In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Qdrant
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.evaluation.qa import QAEvalChain
# from langchain.evaluation.qa import QAGenerateChain

import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file with OpenAI key

#Set API keys for OpenAI and Cohere embeddings
openai.api_key = openai_api_key=os.environ['OPENAI_API_KEY']
cohere_api_key = os.environ['COHERE_API_KEY']

In [2]:
documents=[]
for file in os.listdir('./number_one_menu/final_best_with_GPT/'):
    if 'final_' in file and 'final_website_preprocessed' not in file:
        with open('./number_one_menu/final_best_with_GPT/' + file, 'r', encoding="utf8") as f:
            documents.append(f.read())

In [3]:
# Make some changes in the menu
for ind, doc in enumerate(documents):
    if 'Κλασσική' or 'κλασσική' in doc:
        documents[ind] = doc.replace("Κλασσική", "πίτα" )
        documents[ind] = doc.replace("κλασσική", "πίτα" )

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100) #chunk_size refers to characters, not tokens! Initially set to 1000,200
# Split your documents into texts
texts = text_splitter.create_documents(documents)

#Turn your texts into embeddings
# embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key,model='text-embedding-ada-002') #Alternative option (cheap but not free)
embeddings = CohereEmbeddings(model = "multilingual-22-12", cohere_api_key=cohere_api_key)
#Taken from #https://txt.cohere.com/search-cohere-langchain/
#Cohere Free tier limits: https://docs.cohere.com/docs/going-live
#Information about OpenAI embeddings: https://platform.openai.com/docs/guides/embeddings

#Get your docsearch ready
docsearch = Qdrant.from_documents(texts, embeddings,location=":memory:",  collection_name="my_documents", distance_func="Dot")

Evaluation Chain

In [5]:
#Generates examples in English due to langchain prompt

# example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

# new_examples = example_gen_chain.apply_and_parse(
#     [{"doc": t} for t in documents[:5]]
# )

# new_examples

In [6]:
#Manual examples

examples = [
    {
        "query": "Θα ήθελα να παραγγείλω 3 σουβλάκια κοτόπουλο με σως",
        "answer": "Τα 3 σουβλάκια κοτόπουλο με σως κοστίζουν €6.00"
    },
    {
        "query": "Θα ήθελα μια αμστελ",
        "answer": "Η μπύρα αμστελ 330ml κοστίζει €2.00"
    }
]

In [7]:
#Get predictions
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=openai_api_key,temperature=0, request_timeout=9,max_retries=1, model_name='gpt-3.5-turbo'), 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(k=1), 
    verbose=True,
)

predictions = qa.apply(examples)

print("Predictions \n",predictions)
print("Examples \n",examples)

c:\Users\soyrl\.conda\envs\order_test\lib\site-packages\langchain\llms\openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\soyrl\.conda\envs\order_test\lib\site-packages\langchain\llms\openai.py:739: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(




> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.
Predictions 
 [{'query': 'Θα ήθελα να παραγγείλω 3 σουβλάκια κοτόπουλο με σως', 'answer': 'Τα 3 σουβλάκια κοτόπουλο με σως κοστίζουν €6.00', 'result': 'The cost of each souvlaki chicken with sauce is €2.00. Therefore, the total cost for 3 souvlaki chicken with sauce would be €6.00.'}, {'query': 'Θα ήθελα μια αμστελ', 'answer': 'Η μπύρα αμστελ 330ml κοστίζει €2.00', 'result': "I don't know."}]
Examples 
 [{'query': 'Θα ήθελα να παραγγείλω 3 σουβλάκια κοτόπουλο με σως', 'answer': 'Τα 3 σουβλάκια κοτόπουλο με σως κοστίζουν €6.00'}, {'query': 'Θα ήθελα μια αμστελ', 'answer': 'Η μπύρα αμστελ 330ml κοστίζει €2.00'}]


In [8]:
#Evaluate predictions
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(examples, predictions)
graded_outputs

[{'text': 'CORRECT'}, {'text': 'INCORRECT'}]

In [9]:
#Detailed comparison of predictions with their labels
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])

Example 0:
Question: Θα ήθελα να παραγγείλω 3 σουβλάκια κοτόπουλο με σως
Real Answer: Τα 3 σουβλάκια κοτόπουλο με σως κοστίζουν €6.00
Predicted Answer: The cost of each souvlaki chicken with sauce is €2.00. Therefore, the total cost for 3 souvlaki chicken with sauce would be €6.00.
Predicted Grade: CORRECT
Example 1:
Question: Θα ήθελα μια αμστελ
Real Answer: Η μπύρα αμστελ 330ml κοστίζει €2.00
Predicted Answer: I don't know.
Predicted Grade: INCORRECT


Generate variations of items in menu

In [10]:
# Make each element in the list being an item from the menu - Does not work well with embeddings, needs more context
documents_final=[]
longest_text=0
for text in documents:
    text = text.split('\n')
    for item in text:
        documents_final.append(item) #Get each item of menu as an element of list
        if len(item)>longest_text:
            longest_text=len(item) #Will be ~175 characters

In [11]:
prompt="Παρακάτω δίνεται ένα προϊόν που υπάρχει στο μενού ενός εστιατορίου. Βρες μια εναλλακτική φράση για το προϊόν που μπορεί ένας πελάτης να πει όταν \
θέλει να το παραγγείλει, παραφράζοντάς το όταν είναι δυνατό, ή αλλάζοντας μερικές λεπτομέρειες από αυτό. \
Για παράδειγμα, αντί για '3 σουβλάκια χοιρινά' ο χρήστης μπορεί να πει '3 χοιρινά με σως' και αντί για  \
'halloumi burger (μοσχαρίσιο, χαλούμι, ρόκα, ντομάτα, μαγιονέζα)' μπορεί να πει 'χαλούμι μπέργκερ με ντομάτα και μαγιονέζα'. Απάντησε μόνο με την φράση του χρήστη η οποία \
πρέπει να περικλείεται με << >>. Το προϊόν είναι:"

In [12]:
examples = [{} for i in range(len(documents_final))]

for i in range(20): #~570 items in menu
    retrieved_menu=documents_final[i]
    llm=OpenAI(openai_api_key=openai_api_key,temperature=0.9,model_name='gpt-3.5-turbo')
    response_final=llm.predict(prompt+retrieved_menu) #Predict response using LLM GPT 3.5 turbo
    print("LLM variation:",response_final)
    print("Actual label:",retrieved_menu)
    examples[i]['query']=response_final
    examples[i]['answer']=retrieved_menu

LLM variation: <<Θα ήθελα ένα amstel 330ml, παρακαλώ.>>
Actual label: amstel 330ml - €2.00
LLM variation: <<Θα ήθελα ένα Amstel 500ml παρακαλώ, τιμή 2.40 ευρώ.>>
Actual label: amstel 500ml - €2.40
LLM variation: <<Θα ήθελα μια coca cola 1,5 λίτρου, παρακαλώ.>>
Actual label: coca cola 1500ml - €3.00
LLM variation: <<Μια κόκα κόλα στα 330ml, παρακαλώ.>>
Actual label: coca cola 330ml - €1.60
LLM variation: <<Μια coca cola 500ml, παρακαλώ.>>
Actual label: coca cola 500ml - €1.80
LLM variation: <<Θα ήθελα ένα coca cola light 330ml, παρακαλώ.>>
Actual label: coca cola light 330ml - €1.60
LLM variation: <<Θα ήθελα μια μεγάλη κόκα κόλα light 1500ml, παρακαλώ.>>
Actual label: coca cola zero 1500ml - €3.00
LLM variation: <<Μία zero κόκα κόλα παρακαλώ, 330ml>>
Actual label: coca cola zero 330ml - €1.60
LLM variation: <<Θα ήθελα μια coca cola zero 500ml, παρακαλώ.>>
Actual label: coca cola zero 500ml - €1.80
LLM variation: <<Θα παραγγείλω ένα φαντα λεμονίτα 330ml, παρακαλώ.>>
Actual label: fanta λ

The above variations can also be added to menu to make it easier for LLM to retrieve them when order is placed. Not sure if it helps. Not done yet.